# Wine Quality
### Team 4

https://archive.ics.uci.edu/ml/datasets/Wine+Quality

In [ ]:
import os
from IPython import display
import pandas as pd
import numpy as np
from numpy import arangeG
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support 

In [ ]:
read_df = pd.read_csv('https://raw.githubusercontent.com/AlexHappyCode/wine_quality_project_team_4/main/wine_quality_dataset/winequality-red.csv', ';')
white_df = pd.read_csv('https://raw.githubusercontent.com/AlexHappyCode/wine_quality_project_team_4/main/wine_quality_dataset/winequality-white.csv', ';')
white_df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [ ]:
# Limit read wine to [3, 4, 5, 6, 7, 8]
len(read_df[read_df.quality == 9])

# White wine [3 - 9]
len(white_df[white_df.quality == 10])

0

In [ ]:
y = read_df.iloc[:, 11].values # y is a numpy array
x = read_df.drop(['quality'], axis=1) # dropping the label

# Split the data into trainning set and testing set


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=0.25,
    #random_state=0
    shuffle=False
)

In [ ]:
model = RandomForestClassifier (
    n_estimators=500,
    random_state=42,
    min_samples_split=2,
    min_samples_leaf=1,
    max_depth=10,
    bootstrap=True,
    max_samples=100
)

model.fit(x_train, y_train)

RandomForestClassifier(max_depth=10, max_samples=100, n_estimators=500,
                       random_state=42)

In [ ]:
predict = model.predict(x_test)

In [ ]:
ac = accuracy_score(predict, y_test)
print('Accuracy is:', ac)

Accuracy is: 0.635


# Hyperparameter tuning

The professor said we only need to tune, 2 hyperparameters. I believe that is the number of trees, and the depth of the trees.


In [ ]:
grid_ranges = {
    'n_estimators': [1000],
    'max_depth': np.arange(1, 15, 1),
    'bootstrap': [True],
    'max_samples': [100]
    #'max_depth': [100] # I think this is needed for bootstrap
}
gscv = GridSearchCV(
    estimator=model,
    param_grid=grid_ranges,
    cv=3,
    n_jobs=-1, # how many jobs to run in parallel
    verbose=1,
)
gscv_fit = gscv.fit(x_train, y_train)
best_parameters = gscv_fit.best_params_
print(best_parameters)

Fitting 3 folds for each of 14 candidates, totalling 42 fits
{'bootstrap': True, 'max_depth': 11, 'max_samples': 100, 'n_estimators': 1000}


# Now we can retrain the model with our new hyperparameters

In [ ]:

model = RandomForestClassifier (
    n_estimators=best_parameters['n_estimators'],
    max_depth=best_parameters['max_depth'],
    random_state=42,
    bootstrap=True,
    max_samples=100
)


model.fit(x_train, y_train)


RandomForestClassifier(max_depth=11, max_samples=100, n_estimators=1000,
                       random_state=42)

# Now that we have created our new run time engine with the best hyperparameters, we can test the model

In [ ]:
predict = model.predict(x_test)

In [ ]:
ac = accuracy_score(predict, y_test)
print('Accuracy is:', ac)

Accuracy is: 0.6175


### Calculating Metrics

In [ ]:
scores = precision_recall_fscore_support(predict, y_test)
scores

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0.        , 0.        , 0.67261905, 0.74285714, 0.13793103,
        0.        ]),
 array([0.        , 0.        , 0.66863905, 0.59090909, 0.36363636,
        0.        ]),
 array([0.        , 0.        , 0.67062315, 0.65822785, 0.2       ,
        0.        ]),
 array([  0,   0, 169, 220,  11,   0]))